In [22]:
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np


from core.data import create_records, label_frame

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


En este tutorial aprenderemos a crear `tf.records`; un formato de serializacion de archivos de tensorflow el cual, junto con `tf.Data` permite que el flujo de datos sea eficiente.

In [27]:
datapath = './data/contributions.csv'

contributions = pd.read_csv(datapath)
contributions = contributions.replace({'NaN':'', np.nan:'', 'nan':''})

In [28]:
contrib_uniques.sample(2)

,id,diag_id,ind_id,text,tokens,macro,is_online
31892,34823,ENC_U_4562873419515573861,,realizar la importancia de los adultos mayores,"['realizar', 'importancia', 'adultos', 'mayores']",realizar la importancia de los adultos mayores,0
48398,56765,ENC_U_4614194000564179869,,entregar valores y responsabilidad a los,"['entregar', 'valores', 'responsabilidad']",entregar valores y responsabilidad a los,0


En el archivo `.csv` se encuentran todas las contribuciones asociadas a las encuestas de Dialogos e Individuales. En particular, nosotros trabajaremos con la columna de `text`.


Luego cargamos las etiquetas (ver informe para mas detalles)

In [29]:
labels = pd.read_csv('./data/contribution_labels.csv')
labels.sample(2)

,text,tokens,label
15424,a cuidar el medio ambiente,"['cuidar', 'medio', 'ambiente']",Sustentabilidad Ambiental
7187,empatizar con los otros. unidad. necesitamos b...,"['empatizar', 'unidad', 'necesitamos', 'buenos...",Reciprocidad-Redes


y ocupas este frame para etiquetar la tabla completa 

In [30]:
%%time
cont = label_frame(contributions, labels)

CPU times: user 4min 45s, sys: 15.7 ms, total: 4min 45s
Wall time: 4min 45s


Para entrenar debemos eliminar los duplicados. Ademas solo debemos seleccionar las muestras que estan etiquetadas

In [31]:
contrib_uniques = cont.drop_duplicates(subset='text')
labeled_cont = contrib_uniques[contrib_uniques['label'] != '']

In [35]:
print('objetos etiquetados: {} ({:.2f}%)'.format(labeled_cont.shape[0], 
                                                 labeled_cont.shape[0]/contrib_uniques.shape[0]*100))

objetos etiquetados: 15402 (27.80%)


Antes de ejecutar la funcion debemos cargar nuestro `embedding`. El `embedding` nos dara una representacion vectorial de las palabras. Puedes descargarlo [aqui](http://dcc.uchile.cl/~jperez/word-embeddings/fasttext-sbwc.bin) (4.3GB)

In [7]:
from gensim.models.wrappers import FastText

**WARNING: La operacion siguiente utiliza ~6gb de memoria**

In [8]:
%%time
wordvectors = FastText.load_fasttext_format('./data/fasttext-sbwc.bin')

CPU times: user 2min 17s, sys: 6.11 s, total: 2min 23s
Wall time: 2min 23s


Utilizaremos 3 fold para evaluar nuestro modelo. Ademas debemos filtrar los ejemplos sin etiquetas, ya que estos no nos serviran para el entrenamiento.

In [14]:
%%time
for fold in range(3):
    outdir = './out/records/fold_{}'.format(fold)
    create_records(labeled_cont, wordvectors, outdir, train_ptg=0.5, val_ptg=0.5)

CPU times: user 2min 40s, sys: 712 ms, total: 2min 41s
Wall time: 2min 41s


## Cargando los Records

In [15]:
from core.data import load_records

In [31]:
batches = load_records('./out/records/fold_0/test', batch_size=16, return_all=True)

Cause: could not parse the source code of <function load_records.<locals>.<lambda> at 0x7fea71336c80>: no matching AST found
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function load_records.<locals>.<lambda> at 0x7fea71336c80>: no matching AST found
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [32]:
for b in batches:
    print(b['label'].shape)
    break

(16, 13)


## Guardando datos para predecir

Utilizaremos `create_prediction_record`. Este metodo no guarda etiquetas y tampoco divide el conjunto de entrenamiento.

In [38]:
from core.data import create_prediction_record

In [39]:
%%time
create_prediction_record(cont, wordvectors, outdir = './out/records/')

CPU times: user 5min 30s, sys: 2.64 s, total: 5min 32s
Wall time: 5min 32s


In [40]:
cont.shape

(66701, 8)